In [97]:
import pandas as pd

In [98]:
#%pip install chardet
import chardet

#### Clean and Load H1B Data

In [3]:
data = pd.read_csv("Employer Information 2024.csv", encoding='utf-16', delimiter='\t', quotechar='"')
data.head()

,Line by line,Fiscal Year,Employer (Petitioner) Name,Tax ID,Industry (NAICS) Code,Petitioner City,Petitioner State,Petitioner Zip Code,Initial Approval,Initial Denial,Continuing Approval,Continuing Denial
0,1,2024,NaN,3581.0,"54 - Professional, Scientific, and Technical S...",LAFAYETTE,CA,94549.0,0,0,1,0
1,2,2024,NaN,4245.0,62 - Health Care and Social Assistance,DAVIE,FL,33328.0,1,0,0,0
2,3,2024,1 800 FLOWERS COM INC,7311.0,44-45 - Retail Trade,JERICHO,NY,11753.0,1,0,5,0
3,4,2024,1 HOTEL KAUAI LLC DBA 1 HOTEL HANALEI BAY,5669.0,72 - Accommodation and Food Services,MIAMI,FL,33133.0,0,0,1,0
4,5,2024,1 OAK MEDIA LLC,7999.0,"54 - Professional, Scientific, and Technical S...",LINCOLN CITY,OR,97367.0,1,0,0,0


In [100]:
data.columns

Index(['Line by line', 'Fiscal Year   ', 'Employer (Petitioner) Name',
       'Tax ID', 'Industry (NAICS) Code', 'Petitioner City',
       'Petitioner State', 'Petitioner Zip Code', 'Initial Approval',
       'Initial Denial', 'Continuing Approval', 'Continuing Denial'],
      dtype='object')

In [101]:
data1 = data[["Employer (Petitioner) Name"]]
data1.head()

,Employer (Petitioner) Name
0,NaN
1,NaN
2,1 800 FLOWERS COM INC
3,1 HOTEL KAUAI LLC DBA 1 HOTEL HANALEI BAY
4,1 OAK MEDIA LLC


In [102]:
data1 = data1.rename(columns={
    "Employer (Petitioner) Name": "Employer"
})

In [103]:
data1.head()

,Employer
0,NaN
1,NaN
2,1 800 FLOWERS COM INC
3,1 HOTEL KAUAI LLC DBA 1 HOTEL HANALEI BAY
4,1 OAK MEDIA LLC


In [104]:
data1.isna().value_counts()

Employer
False       61454
True            2
Name: count, dtype: int64

In [105]:
data1 = data1.dropna()

In [106]:
#data1[data1["Employer"].duplicated()]
data1[data1["Employer"].duplicated(keep=False)]

,Employer
22,1UPHEALTH INC
23,1UPHEALTH INC
45,2U INC
46,2U INC
50,3 GIS LLC
...,...
61410,ZUME IT INC
61425,ZURU LLC
61426,ZURU LLC
61453,ZYTUS INC


In [107]:
data1 = data1.drop_duplicates(subset="Employer").reset_index(drop=True)

In [108]:
data1

,Employer
0,1 800 FLOWERS COM INC
1,1 HOTEL KAUAI LLC DBA 1 HOTEL HANALEI BAY
2,1 OAK MEDIA LLC
3,1 POINT SYSTEM LLC
4,1 REALTY GLOBAL INC
...,...
52887,ZYNO MEDICAL LLC
52888,ZYON DIGITAL SOLUTIONS LLC
52889,ZYSCOVICH LLC
52890,ZYTUS INC


In [109]:
data1.to_csv("employer2024.csv", index=False)

#### Scrape Entry Level 'Data Analyst' Job Postings in California Within 12 Hours from Linkedin

In [99]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd
from selenium.common.exceptions import ElementClickInterceptedException, NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [32]:
#def setup_driver():
#    options = webdriver.ChromeOptions()
#    #options.add_argument("--headless")
#    options.add_argument("--no-sandbox")
#    options.add_argument("--disable-dev-shm-usage")
#    return webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

In [100]:
# update
def setup_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--user-data-dir=C:/Users/tcjen/AppData/Local/Google/Chrome/User Data/Default")
    options.add_argument("--profile-directory=Default")  
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.maximize_window()
    return driver

In [83]:
#update2
from selenium import webdriver
from selenium.webdriver.edge.service import Service as EdgeService
from selenium.webdriver.edge.options import Options as EdgeOptions
from webdriver_manager.microsoft import EdgeChromiumDriverManager

def setup_driver():
    options = EdgeOptions()
    options.use_chromium = True
    # options.add_argument("--headless")
    options.add_argument("--start-maximized")  # 🪟 Opens full screen
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")

    driver = webdriver.Edge(service=EdgeService(EdgeChromiumDriverManager().install()), options=options)
    return driver

In [101]:
def close_sign_in_modal(driver):
    try:
        # Find and click the "X" button with aria-label="Dismiss"
        close_btn = driver.find_element(By.XPATH, "//div[contains(@class, 'modal__overlay') and contains(@class, 'modal__overlay--visible')]//button[@aria-label='Dismiss']")
        driver.execute_script("arguments[0].click();", close_btn)
        print("Sign-in modal closed.")
        time.sleep(1)  # Short pause to allow modal to disappear
    except Exception as e:
        print("No sign-in modal to close or failed:", e)

In [102]:
def scroll_to_load_jobs(driver, pause_time=2, max_scrolls=60):
    for i in range(max_scrolls):
        try:
            # Scroll to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(pause_time)

            # Try clicking "See more jobs" if it's there
            see_more = driver.find_element(By.CLASS_NAME, "infinite-scroller__show-more-button")
            if see_more.is_displayed():
                print(f"Clicking 'See more jobs' (scroll {i+1})")
                see_more.click()
                time.sleep(pause_time)
        except (NoSuchElementException, ElementClickInterceptedException):
            print(f"No more 'See more jobs' button after scroll {i+1}")
            break

In [24]:
#def fetch_job_elements(driver):
#    return driver.find_elements(By.XPATH, "//div[contains(@class, 'base-search-card__info')]")

In [103]:
# updated
def fetch_job_elements(driver):
    return driver.find_elements(By.XPATH, "//div[contains(@class, 'base-card')]")

In [104]:
def extract_job_info(job_card):
    try:
        title = job_card.find_element(By.XPATH, ".//a[contains(@class, 'job-card-list__title--link')]//strong").text
    except:
        title = "N/A"
    try:
        company = job_card.find_element(By.XPATH, "//div[contains(@class, 'artdeco-entity-lockup__subtitle')]//span").text
    except:
        company = "N/A"
    try:
        location = job_card.find_element(By.XPATH, ".//ul[contains(@class, 'job-card-container__metadata-wrapper')]//span").text
    except:
        location = "N/A"
    try:
        posted_time = job_card.find_element(By.XPATH, ".//time").text
    except:
        posted_time = "N/A"
    return title, company, location, posted_time

In [105]:
def extract_applicants_from_right_pane(driver):
    try:
        elements = driver.find_elements(By.XPATH, "//*[contains(translate(text(), 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz'), 'applicant')]")
        for element in elements:
            text = element.text.strip()
            if text and "applicant" in text.lower():
                return text
        return "N/A"
    except:
        return "N/A"

In [28]:
#def get_linkedin_jobs():
#    url = "https://www.linkedin.com/jobs/search/?keywords=data%20scientist&location=California&f_TPR=r43200"
#    driver = setup_driver()
#    driver.get(url)
#    time.sleep(3)

#    close_sign_in_modal(driver)
    
#    scroll_to_load_jobs(driver, pause_time=2, max_scrolls=10)

#    job_cards = fetch_job_elements(driver)
#    print(f"Found {len(job_cards)} job cards")

#    job_data = [extract_job_info(card) for card in job_cards]

    #driver.quit()
#    return pd.DataFrame(job_data, columns=["Position", "Company", "Location", "Posted Time"])

In [106]:
def inject_linkedin_cookies(driver):
    # Load LinkedIn homepage to set cookie domain
    driver.get("https://www.linkedin.com")
    time.sleep(2)

    cookies = [
        {"name": "li_at", "value": "AQEDATrAam8EAMd2AAABly1qL6QAAAGXUXazpFYAGyyxIQY-GLZa62VggSDXBXIx8Liaz1K13mXRGDF40eSSHOtmVplSiHMDol-b2HiBjy646hcMP1C62e8y54Wb81cBE773I8XHgWp6hPF9VPL4vBZV", "domain": ".linkedin.com"},
        {"name": "JSESSIONID", "value": "ajax:0440216602495823794", "domain": ".linkedin.com"},
        {"name": "bcookie", "value": "v=2&8ff0ab71-d35a-4931-8fd6-1891aec9fbd2", "domain": ".linkedin.com"},
    ]

    for cookie in cookies:
        driver.add_cookie(cookie)

    print("✅ Cookies injected successfully")


In [107]:
# updated 2
def get_linkedin_jobs():
    url = "https://www.linkedin.com/jobs/search/?keywords=data%20scientist&location=california&f_TPR=r43200"
    driver = setup_driver()
    #driver.delete_all_cookies()
    #inject_linkedin_cookies(driver)
    driver.get(url)
    time.sleep(3)

    close_sign_in_modal(driver)
    scroll_to_load_jobs(driver, pause_time=2, max_scrolls=60)

    job_cards = fetch_job_elements(driver)
    print(f"Found {len(job_cards)} job cards")

    job_data = []

    for i in range(len(job_cards)):
        print(f"\n📄 Processing job {i+1}/{len(job_cards)}")

        try:
            # Re-fetch to avoid stale element references
            job_cards = fetch_job_elements(driver)
            card = job_cards[i]

            # Extract basic info from the card
            title, company, location#, posted_time = extract_job_info(card)

            # Get job URL from the anchor tag
            job_link = card.find_element(By.XPATH, ".//a[contains(@class, 'base-card__full-link')]")
            job_url = job_link.get_attribute("href")

            # Open the job post in a new tab
            #driver.execute_script("window.open(arguments[0]);", job_url)
            #driver.switch_to.window(driver.window_handles[-1])
            #time.sleep(2)

            # Extract applicants from full job page
            #applicants = extract_applicants_from_right_pane(driver)
            #print(f"👤 Applicants: {applicants}")

            # Close job tab and switch back to main tab
            #driver.close()
            #driver.switch_to.window(driver.window_handles[0])

        except Exception as e:
            print(f"❌ Failed to process job {i+1}: {e}")
            applicants = "N/A"
            job_url = "N/A"
            title = title if 'title' in locals() else "N/A"
            company = company if 'company' in locals() else "N/A"
            location = location if 'location' in locals() else "N/A"
            posted_time = posted_time if 'posted_time' in locals() else "N/A"

        job_data.append((title, company, location, posted_time, applicants, job_url))
        #job_data.append((title, company, location, job_url))

    #driver.quit()
    return pd.DataFrame(job_data, columns=["Position", "Company", "Location", "Posted Time", "Applicants", "Job URL"])
    #return pd.DataFrame(job_data, columns=["Position", "Company", "Location", "Job URL"])

In [108]:
df_jobs = get_linkedin_jobs()

No sign-in modal to close or failed: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[contains(@class, 'modal__overlay') and contains(@class, 'modal__overlay--visible')]//button[@aria-label='Dismiss']"}
  (Session info: chrome=137.0.7151.55); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x0x76f173+62931]
	GetHandleVerifier [0x0x76f1b4+62996]
	(No symbol) [0x0x581053]
	(No symbol) [0x0x5c97ae]
	(No symbol) [0x0x5c9b4b]
	(No symbol) [0x0x6121f2]
	(No symbol) [0x0x5ee4f4]
	(No symbol) [0x0x60f9e3]
	(No symbol) [0x0x5ee2a6]
	(No symbol) [0x0x5bd5f0]
	(No symbol) [0x0x5be464]
	GetHandleVerifier [0x0x9c3463+2504899]
	GetHandleVerifier [0x0x9be892+2485490]
	GetHandleVerifier [0x0x79590a+220522]
	GetHandleVerifier [0x0x786388+157672]
	GetHandleVerifier [0x0x78cb6d+184269]
	GetHandleVerifier [0x0x777158+95672]
	GetHandleVerifi

In [275]:
df_jobs.head(10)

,Position,Company,Location,Posted Time,Applicants,Job URL
0,"Data Scientist- TikTok Ads, Privacy",TikTok,"San Jose, CA",9 hours ago,Over 200 applicants,https://www.linkedin.com/jobs/view/data-scient...
1,Machine Learning Engineer,Acubed,"Sunnyvale, CA",10 hours ago,150 applicants,https://www.linkedin.com/jobs/view/machine-lea...
2,Machine Learning Engineer - TikTok Search Busi...,TikTok,"San Jose, CA",10 hours ago,89 applicants,https://www.linkedin.com/jobs/view/machine-lea...
3,"Software Engineer, ML Infrastructure, Limited ...",Waymo,"Mountain View, CA",8 hours ago,45 applicants,https://www.linkedin.com/jobs/view/software-en...
4,"DATA SCIENTIST, PRODUCT",TieTalent,"California, United States",10 hours ago,Be among the first 25 applicants,https://www.linkedin.com/jobs/view/data-scient...
5,Machine Learning Engineer - Trust and Safety,TikTok,"San Jose, CA",10 hours ago,165 applicants,https://www.linkedin.com/jobs/view/machine-lea...
6,Data Scientist - Growth/Marketing,TieTalent,"California, United States",11 hours ago,Be among the first 25 applicants,https://www.linkedin.com/jobs/view/data-scient...
7,Machine Learning Engineer,TieTalent,"California, United States",11 hours ago,N/A,https://www.linkedin.com/jobs/view/machine-lea...
8,Machine Learning Engineer,TieTalent,"California, United States",11 hours ago,Be among the first 25 applicants,https://www.linkedin.com/jobs/view/machine-lea...
9,Machine Learning Engineer,TieTalent,"California, United States",11 hours ago,Be among the first 25 applicants,https://www.linkedin.com/jobs/view/machine-lea...


In [276]:
df_jobs['Applicants'].value_counts()

Applicants
Be among the first 25 applicants    46
N/A                                  8
Over 200 applicants                  6
45 applicants                        5
86 applicants                        3
29 applicants                        2
109 applicants                       2
67 applicants                        2
190 applicants                       2
46 applicants                        2
36 applicants                        2
150 applicants                       1
89 applicants                        1
39 applicants                        1
181 applicants                       1
38 applicants                        1
143 applicants                       1
165 applicants                       1
147 applicants                       1
26 applicants                        1
138 applicants                       1
Name: count, dtype: int64

In [277]:
df_jobs.to_csv("linkedin_jobs.csv", index=False)

#### Compare With H1B Data

In [278]:
# Load the CSVs
employers = pd.read_csv('employer2024.csv') 
jobs = pd.read_csv('linkedin_jobs.csv')       

In [279]:
jobs.head()

,Position,Company,Location,Posted Time,Applicants,Job URL
0,"Data Scientist- TikTok Ads, Privacy",TikTok,"San Jose, CA",9 hours ago,Over 200 applicants,https://www.linkedin.com/jobs/view/data-scient...
1,Machine Learning Engineer,Acubed,"Sunnyvale, CA",10 hours ago,150 applicants,https://www.linkedin.com/jobs/view/machine-lea...
2,Machine Learning Engineer - TikTok Search Busi...,TikTok,"San Jose, CA",10 hours ago,89 applicants,https://www.linkedin.com/jobs/view/machine-lea...
3,"Software Engineer, ML Infrastructure, Limited ...",Waymo,"Mountain View, CA",8 hours ago,45 applicants,https://www.linkedin.com/jobs/view/software-en...
4,"DATA SCIENTIST, PRODUCT",TieTalent,"California, United States",10 hours ago,Be among the first 25 applicants,https://www.linkedin.com/jobs/view/data-scient...


In [280]:
# Clean and lowercase for better matching
employers = employers['Employer'].dropna().str.lower().str.strip().tolist()
jobs['company_clean'] = jobs['Company'].fillna('').str.lower().str.strip()

# Define a function to check if a company name is contained in any employer name
def is_rough_match(company_name):
    return any(company_name in employer for employer in employers)

# Apply the function
filtered_jobs = jobs[jobs['company_clean'].apply(is_rough_match)]

# Drop the helper column if you want
filtered_jobs = filtered_jobs.drop(columns=['company_clean'])
filtered_jobs = filtered_jobs.reset_index(drop=True)

In [281]:
filtered_jobs

,Position,Company,Location,Posted Time,Applicants,Job URL
0,"Data Scientist- TikTok Ads, Privacy",TikTok,"San Jose, CA",9 hours ago,Over 200 applicants,https://www.linkedin.com/jobs/view/data-scient...
1,Machine Learning Engineer,Acubed,"Sunnyvale, CA",10 hours ago,150 applicants,https://www.linkedin.com/jobs/view/machine-lea...
2,Machine Learning Engineer - TikTok Search Busi...,TikTok,"San Jose, CA",10 hours ago,89 applicants,https://www.linkedin.com/jobs/view/machine-lea...
3,"Software Engineer, ML Infrastructure, Limited ...",Waymo,"Mountain View, CA",8 hours ago,45 applicants,https://www.linkedin.com/jobs/view/software-en...
4,Machine Learning Engineer - Trust and Safety,TikTok,"San Jose, CA",10 hours ago,165 applicants,https://www.linkedin.com/jobs/view/machine-lea...
5,"Machine Learning Engineer, Search Engine, E-Co...",TikTok,"San Jose, CA",10 hours ago,38 applicants,https://www.linkedin.com/jobs/view/machine-lea...
6,Machine Learning Engineer - Search Engine - E-...,TikTok,"San Jose, CA",9 hours ago,Be among the first 25 applicants,https://www.linkedin.com/jobs/view/machine-lea...
7,Machine Learning Engineer - CV/Multimodal (Tik...,TikTok,"San Jose, CA",10 hours ago,NaN,https://www.linkedin.com/jobs/view/machine-lea...
8,Software Development Engineer,Rubrik,"Palo Alto, CA",6 hours ago,143 applicants,https://www.linkedin.com/jobs/view/software-de...
9,Software Engineer in ML Engineering Platform,ByteDance,"San Jose, CA",10 hours ago,86 applicants,https://www.linkedin.com/jobs/view/software-en...


In [282]:
def get_matching_employer(company_name):
    for employer in employers:
        if company_name in employer:
            return employer  # Return the first match
    return None  # No match found

# Apply the function to get the matching employer name
jobs['matched_employer'] = jobs['company_clean'].apply(get_matching_employer)

# Filter only the rows where a match was found
filtered_jobs = jobs[jobs['matched_employer'].notna()]

In [283]:
filtered_jobs.columns

Index(['Position', 'Company', 'Location', 'Posted Time', 'Applicants',
       'Job URL', 'company_clean', 'matched_employer'],
      dtype='object')

In [284]:
filtered_jobs = filtered_jobs[['Position', 'Company', 'matched_employer', 'Location', 'Posted Time', 'Applicants', 'Job URL']]
filtered_jobs = filtered_jobs.reset_index(drop=True)

In [285]:
filtered_jobs

,Position,Company,matched_employer,Location,Posted Time,Applicants,Job URL
0,"Data Scientist- TikTok Ads, Privacy",TikTok,tiktok inc,"San Jose, CA",9 hours ago,Over 200 applicants,https://www.linkedin.com/jobs/view/data-scient...
1,Machine Learning Engineer,Acubed,datacubed llc,"Sunnyvale, CA",10 hours ago,150 applicants,https://www.linkedin.com/jobs/view/machine-lea...
2,Machine Learning Engineer - TikTok Search Busi...,TikTok,tiktok inc,"San Jose, CA",10 hours ago,89 applicants,https://www.linkedin.com/jobs/view/machine-lea...
3,"Software Engineer, ML Infrastructure, Limited ...",Waymo,waymo llc,"Mountain View, CA",8 hours ago,45 applicants,https://www.linkedin.com/jobs/view/software-en...
4,Machine Learning Engineer - Trust and Safety,TikTok,tiktok inc,"San Jose, CA",10 hours ago,165 applicants,https://www.linkedin.com/jobs/view/machine-lea...
5,"Machine Learning Engineer, Search Engine, E-Co...",TikTok,tiktok inc,"San Jose, CA",10 hours ago,38 applicants,https://www.linkedin.com/jobs/view/machine-lea...
6,Machine Learning Engineer - Search Engine - E-...,TikTok,tiktok inc,"San Jose, CA",9 hours ago,Be among the first 25 applicants,https://www.linkedin.com/jobs/view/machine-lea...
7,Machine Learning Engineer - CV/Multimodal (Tik...,TikTok,tiktok inc,"San Jose, CA",10 hours ago,NaN,https://www.linkedin.com/jobs/view/machine-lea...
8,Software Development Engineer,Rubrik,rubrik inc,"Palo Alto, CA",6 hours ago,143 applicants,https://www.linkedin.com/jobs/view/software-de...
9,Software Engineer in ML Engineering Platform,ByteDance,bytedance inc,"San Jose, CA",10 hours ago,86 applicants,https://www.linkedin.com/jobs/view/software-en...


#### Extract Applicants Count

In [286]:
import re
def extract_applicant_count(text):
    text = str(text).lower().strip()
    
    if "first" in text:
        match = re.search(r'first (\d+)', text)
        return int(match.group(1)) if match else None
    elif "over" in text:
        match = re.search(r'over (\d+)', text)
        return int(match.group(1))  # or return a high number like 999 if you want to always exclude
    else:
        match = re.search(r'(\d+)', text)
        return int(match.group(1)) if match else None

In [287]:
# Apply the function
filtered_jobs['applicant_count'] = filtered_jobs['Applicants'].apply(extract_applicant_count)

# Filter where count is less than 80
filtered_jobs = filtered_jobs[filtered_jobs['applicant_count'] < 80]

# Optionally drop the helper column
filtered_jobs = filtered_jobs.drop(columns=['applicant_count'])
filtered_jobs = filtered_jobs.reset_index(drop=True)

In [288]:
filtered_jobs

,Position,Company,matched_employer,Location,Posted Time,Applicants,Job URL
0,"Software Engineer, ML Infrastructure, Limited ...",Waymo,waymo llc,"Mountain View, CA",8 hours ago,45 applicants,https://www.linkedin.com/jobs/view/software-en...
1,"Machine Learning Engineer, Search Engine, E-Co...",TikTok,tiktok inc,"San Jose, CA",10 hours ago,38 applicants,https://www.linkedin.com/jobs/view/machine-lea...
2,Machine Learning Engineer - Search Engine - E-...,TikTok,tiktok inc,"San Jose, CA",9 hours ago,Be among the first 25 applicants,https://www.linkedin.com/jobs/view/machine-lea...
3,"Research Scientist, Data Management and Securi...",ByteDance,bytedance inc,"San Jose, CA",10 hours ago,Be among the first 25 applicants,https://www.linkedin.com/jobs/view/research-sc...
4,Jr. Mobile Developer,Tech Consulting,"alfa tech consulting engineers, inc.","California, United States",6 hours ago,67 applicants,https://www.linkedin.com/jobs/view/jr-mobile-d...
5,Platform Engineer,Aptos Labs,matonee inc d b a aptos labs,"Palo Alto, CA",6 hours ago,Be among the first 25 applicants,https://www.linkedin.com/jobs/view/platform-en...
6,Software Development Engineer in Test,Align Technology,align technology inc,"San Jose, CA",3 hours ago,46 applicants,https://www.linkedin.com/jobs/view/software-de...
7,Frontend Software Engineer - PGC,TikTok,tiktok inc,"San Jose, CA",9 hours ago,Be among the first 25 applicants,https://www.linkedin.com/jobs/view/frontend-so...
8,Associate QA Engineer,Veeva Systems,veeva systems inc,"Pleasanton, CA",8 hours ago,36 applicants,https://www.linkedin.com/jobs/view/associate-q...
9,Technical Solutions Engineer,Epic,edgewood partners insurance center d b a epic ...,"Sacramento, CA",5 hours ago,45 applicants,https://www.linkedin.com/jobs/view/technical-s...


In [289]:
filtered_jobs.to_csv("final.csv", index=False)

## test

In [49]:
def extract_applicants_from_right_pane(driver):
    try:
        elements = driver.find_elements(By.XPATH, "//*[contains(translate(text(), 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz'), 'applicant')]")
        for element in elements:
            text = element.text.strip()
            if text and "applicant" in text.lower():
                return text
        return "N/A"
    except:
        return "N/A"

In [50]:
def fetch_job_elements(driver):
    return driver.find_elements(By.XPATH, "//div[contains(@class, 'base-card')]")

In [51]:
driver = setup_driver()
driver.get("https://www.linkedin.com/jobs/search/?keywords=data%20scientist&location=California&f_TPR=r43200")
time.sleep(5)
close_sign_in_modal(driver)

No sign-in modal to close or failed: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[contains(@class, 'modal__overlay') and contains(@class, 'modal__overlay--visible')]//button[@aria-label='Dismiss']"}
  (Session info: chrome=136.0.7103.93); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x0108FC53+61635]
	GetHandleVerifier [0x0108FC94+61700]
	(No symbol) [0x00EB05D3]
	(No symbol) [0x00EF899E]
	(No symbol) [0x00EF8D3B]
	(No symbol) [0x00F40E12]
	(No symbol) [0x00F1D2E4]
	(No symbol) [0x00F3E61B]
	(No symbol) [0x00F1D096]
	(No symbol) [0x00EEC840]
	(No symbol) [0x00EED6A4]
	GetHandleVerifier [0x01314573+2701795]
	GetHandleVerifier [0x0130FCF6+2683238]
	GetHandleVerifier [0x0132AA3E+2793134]
	GetHandleVerifier [0x010A6915+155013]
	GetHandleVerifier [0x010ACFFD+181357]
	GetHandleVerifier [0x010974A8+92440]
	GetHandleVerif

In [52]:
scroll_to_load_jobs(driver, pause_time=2, max_scrolls=2)

In [53]:
# Re-fetch all job cards to get the latest DOM snapshot
job_cards = fetch_job_elements(driver)

# Choose the index of the job you want to test (e.g., 2 = third job)
index_to_test = 0

try:
    card = job_cards[index_to_test]

    # Find the correct clickable link inside the card
    job_link = card.find_element(By.XPATH, ".//a[contains(@class, 'base-card__full-link')]")
    driver.execute_script("arguments[0].scrollIntoView();", job_link)
    driver.execute_script("arguments[0].click();", job_link)
    print(f"✅ Clicked job card at index {index_to_test}")
    time.sleep(2)

    # Test applicant extraction
    applicants = extract_applicants_from_right_pane(driver)
    print(f"👤 Applicants: {applicants}")

except Exception as e:
    print(f"❌ Failed to process job at index {index_to_test}: {e}")

✅ Clicked job card at index 0
👤 Applicants: Be an early applicant
